# Austin, TX Crime Analysis

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from usefuls import atx_zip_codes, census_key, zipcode_tabulation_area, similar_offense_type

$$\sum_{w\in\mathfrak{S}_n}{q^{\text{inv}(w)}} = [n]_q!$$

## Get Property Data from Zillow
* The data file is too large to upload to GitHub, but it is obtainable from [here](https://www.kaggle.com/zillow/zecon#Zip_time_series.cs)

In [4]:
# Read the file
zillow_df = pd.read_csv('Zip_time_series.csv')

# Rename RegionName to zipcode
zillow_df = zillow_df.rename(columns={'RegionName': 'Zip Code'})

# Filter out all non-Austin area zipcodes
zillow_df = zillow_df[[(z in atx_zip_codes) for z in zillow_df['Zip Code']]]

# Add year column to dataframe
zillow_df['year'] = zillow_df['Date'].apply(lambda s : int(s[:4]))

# Filter out all years before 2009
zillow_df = zillow_df[(zillow_df['year'] >= 2011) & (zillow_df['year'] <= 2016)]

# Keep only the columns we need
zillow_df = zillow_df[['year', 'Zip Code', 'ZHVI_AllHomes']]

# Drop NaN values
zillow_df = zillow_df.dropna()

# Group data by year, zipcode, find the mean ZHVI per year per zipcode
zillow_df = round(zillow_df.groupby(['year', 'Zip Code']).ZHVI_AllHomes.mean(),2).to_frame()

# Preview the frame
zillow_df.head()

ZHVI_AllHomes
year Zip Code               
2011 78610         167591.67
     78613         182450.00
     78617         104841.67
     78641         143600.00
     78664         130350.00

In [5]:
zillow_df

ZHVI_AllHomes
year Zip Code               
2011 78610         167591.67
     78613         182450.00
     78617         104841.67
     78641         143600.00
     78664         130350.00
     78681         191216.67
     78701         303008.33
     78702         198791.67
     78703         509408.33
     78704         301783.33
     78717         230816.67
     78721         122766.67
     78722         240833.33
     78723         175683.33
     78725          99683.33
     78726         323016.67
     78727         181725.00
     78728         159008.33
     78729         175908.33
     78731         427883.33
     78733         458083.33
     78735         317616.67
     78736         210675.00
     78738         427900.00
     78739         317150.00
     78741         107750.00
     78744          99566.67
     78745         157891.67
     78746         616233.33
     78748         167125.00
...                      ...
2016 78703         749500.00
     78704         487333.33
     78717         317883.33
     78721         237050.00
     78722         388491.67
     78723         312991.67
     78725         148666.67
     78726         427533.33
     78727         270716.67
     78728         233358.33
     78729         260083.33
     78731         600833.33
     78733         625216.67
     78735         438458.33
     78736         301666.67
     78738         515550.00
     78739         422591.67
     78741         177433.33
     78744         164308.33
     78745         253033.33
     78746         869716.67
     78748         242100.00
     78749         315800.00
     78751         398283.33
     78752         268475.00
     78753         192108.33
     78756         458066.67
     78757         378691.67
     78758         218191.67
     78759         402408.33

[228 rows x 1 columns]